# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hradyz'k,49.2346,33.1361,32.74,35,0,8.19,UA,1691312064
1,1,waitangi,-43.9535,-176.5597,7.68,60,100,6.83,NZ,1691312064
2,2,puerto natales,-51.7236,-72.4875,-0.89,75,97,7.14,CL,1691312064
3,3,olonkinbyen,70.9221,-8.7187,3.35,97,100,9.82,SJ,1691312065
4,4,tara,56.8944,74.3710,25.54,35,0,4.10,RU,1691312065


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=600,
    frame_height=400,
    size = "Humidity",
    scale = .75,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_weather = city_data_df[(city_data_df["Humidity"] < 10)]

# Drop any rows with null values
no_na_weather = filtered_weather.dropna()

# Display sample data
no_na_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,ghat,24.9647,10.1728,39.22,9,0,4.80,LY,1691312070
56,56,taoudenni,22.6783,-3.9836,40.87,9,0,6.54,ML,1691312075
133,133,shihezi,44.3000,86.0333,40.60,9,8,4.16,CN,1691312089
163,163,medina,24.4686,39.6142,39.24,5,0,7.72,SA,1691311884
301,301,tarinkot,32.6268,65.8733,37.70,6,0,6.19,AF,1691312122
335,335,nushki,29.5542,66.0215,39.84,8,5,5.40,PK,1691312129
354,354,ad dilam,23.9915,47.1627,39.23,7,7,8.10,SA,1691312132
362,362,illizi,26.4833,8.4667,36.89,7,0,3.19,DZ,1691312135
397,397,isfahan,32.6572,51.6776,34.80,8,0,2.04,IR,1691312141
525,525,chaman,30.9236,66.4512,37.77,7,0,5.87,PK,1691312166


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
selected_columns = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = no_na_weather[selected_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
32,ghat,LY,24.9647,10.1728,9,
56,taoudenni,ML,22.6783,-3.9836,9,
133,shihezi,CN,44.3000,86.0333,9,
163,medina,SA,24.4686,39.6142,5,
301,tarinkot,AF,32.6268,65.8733,6,
335,nushki,PK,29.5542,66.0215,8,
354,ad dilam,SA,23.9915,47.1627,7,
362,illizi,DZ,26.4833,8.4667,7,
397,isfahan,IR,32.6572,51.6776,8,
525,chaman,PK,30.9236,66.4512,7,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "limit": 1,
          "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ghat - nearest hotel: فندق تاسيلي
taoudenni - nearest hotel: No hotel found
shihezi - nearest hotel: 石河子宾馆
medina - nearest hotel: The Oberoi Madina
tarinkot - nearest hotel: No hotel found
nushki - nearest hotel: No hotel found
ad dilam - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
isfahan - nearest hotel: هتل فیروزه
chaman - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
32,ghat,LY,24.9647,10.1728,9,فندق تاسيلي
56,taoudenni,ML,22.6783,-3.9836,9,No hotel found
133,shihezi,CN,44.3000,86.0333,9,石河子宾馆
163,medina,SA,24.4686,39.6142,5,The Oberoi Madina
301,tarinkot,AF,32.6268,65.8733,6,No hotel found
335,nushki,PK,29.5542,66.0215,8,No hotel found
354,ad dilam,SA,23.9915,47.1627,7,No hotel found
362,illizi,DZ,26.4833,8.4667,7,بوناقة
397,isfahan,IR,32.6572,51.6776,8,هتل فیروزه
525,chaman,PK,30.9236,66.4512,7,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=600,
    frame_height=400,
    size = "Humidity",
    scale = 5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)